<a href="https://colab.research.google.com/github/tomgoter/nlp_finalproject/blob/master/bert_xlnet_uda_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT and XLNet Fine-Tuned Notebook with UDA Options
## W266 Final Project
### Game of Thrones Text Classification
### T. P. Goter
### Fall 2019

This notebook is used to perform the baseline, finetuned BERT supervised text classification. The original UDA process utilized a Python script wrapped in a bash shell script. This notebook was generated in order to better show and annotate the process.

## Acknowledgement
Much of this code was leveraged from the open source [UDA](https://github.com/google-research/uda). It has been adapted to the Game of Thrones dataset. 

## Import Data Libraries

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import os
import collections
import tensorflow as tf
#tf.enable_eager_execution()
print(tf.__version__)

import yaml
import pprint

from absl import app
from absl import logging

# Mounting the drive is straightforward but required authentication each time 
# we reset the session
from google.colab import drive

drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/NLP_FinalProject/working'


os.listdir(drive_path)



#
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime!')
  tpu_address = False
else:
  from google.colab import auth
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  bucket_address = "gs://nlp_finalproject"
  assert bucket_address, 'Must specify an existing GCS bucket name'
  print('Using bucket: {}'.format(bucket_address))

  auth.authenticate_user()
  
  # Upload credentials to TPU.
  with tf.Session(tpu_address) as sess:    
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)
 






1.15.0
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
TPU address is grpc://10.34.225.66:8470
Using bucket: gs://nlp_finalproject
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_De

## Get BERT onto Google Drive - DO THIS ONCE


In [0]:
#! mkdir /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained
# download bert base
#! wget -P /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
#! unzip /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/uncased_L-12_H-768_A-12.zip &&
#! rm /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/uncased_L-12_H-768_A-12.zip
#! mkdir /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/bert_base
#! mv uncased_L-12_H-768_A-12 /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/bert_base
#! mv /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/bert_base/uncased*/* /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/bert_base/
#os.listdir(drive_path + '/bert_pretrained/bert_base/')

## Update Repository


In [0]:
!git -C /content/drive/My\ Drive/NLP_FinalProject/working pull

Already up to date.


## Create symbolic links to the Google Drive directories

In [0]:
!ln -s '/content/drive/My Drive/NLP_FinalProject/working/bert' bpath 
!ln -s '/content/drive/My Drive/NLP_FinalProject/working/' wpath 
!ln -s '/content/drive/My Drive/NLP_FinalProject/working/utils' upath 
!ln -s '/content/drive/My Drive/NLP_FinalProject/working/xlnet' xpath 

In [0]:
from bpath import modeling
from wpath import uda_colab
from upath import proc_data_utils, raw_data_utils, model_utils
from xpath import xlnet_colab

## Define Some Options
This section replaces passing the input parameters as command line arguments. This section is very important. It controls the entire model. See the dictionary below.

### Task Options:
- **do_train:** Boolean of whether we are training
- **do_eval:** Boolean of whether we are just evaluating

### Training Options:
- **sup_train_data_dir:** Input directory for supervised data. This should be set to "./Data/proc_data/train_##" where the ## is one of the subsets of training data generated from the prepro_ALL.csh script.
- **eval_data_dir:**  The input data dir of the evaluation data. This should be the path to the development data with which we will do hyperparameter tuning. We can change this to the test data directory once we are ready for final evaluation. The dev data path is: "./Data/proc_data/dev"
- **unsup_data_dir:** The input data dir of the unsupervised data. Path for the unsupervised, augmented data. This should be equal to "./Data/proc_data/unsup"
- **bert_config_file:** Absolute path to the json file corresponding to the pre-trained BERT model. For us this is: "./bert_pretrained/bert_base/bert_config.json"
- **vocab_file:** The vocabulary file that the BERT model was trained on. This should be equal to "./bert_pretrained/bert_base/vocab.txt"
- **init_checkpoint:** Initial checkpoint from the pre-trained BERT model. This should be equal to: "./bert_pretrained/bert_base/bert_model.ckpt"
- **task_name:** The name of the task to train. This should be equal to "GoT"
- **model_dir:** The output directory where the model checkpoints will be written. This will be set to "models" followed by a case specific identifier.

### Model configuration
- **use_one_hot_embeddings:** Boolean, default: True, If True, tf.one_hot will be used for embedding lookups, otherwise tf.nn.embedding_lookup will be used. On TPUs, this should be True since it is much faster."
- **max_seq_length":** Integer, default = 128, The maximum total sequence length after WordPiece tokenization. Sequences longer than this will be truncated, and sequences shorter than this will be padded. Note, GoT data was processed to be on-average close to this length to minimize lost data.
- **model_dropout:** Float, default = -1 (i.e., no dropout). Dropout rate for both the attention and the hidden states.

### Training hyper-parameters
- **train_batch_size:** Integer, default = 32. Based on the discussion [here](https://github.com/google-research/bert#out-of-memory-issues). 32 is probably the largest we can run with 11 GB of RAM while using BERT base with a maximum sequence length of 128.
- **eval_batch_size:** Integer, default = 8, "Base batch size for evaluation."
- **save_checkpoints_num:** Integer, default = 20, Number of checkpoints to save during training.
- **iterations_per_loop:** Integer, default = 200, Number of steps to make in each estimator call.
- **num_train_steps:** Integer, no default, number of training steps

### Optimizer hyperparameters
- **learning_rate:** Float, default = 2e-5, The initial learning rate for Adam Optimizer
- **num_warmup_steps:** Integer, no default, Number of warmup steps
- **clip_norm:** Float, default= 1.0, Gradient clip hyperparameter.

### UDA Options:
- **unsup_ratio:** Integer - ratio between unsupervised batch size and supervised batch size. If zero - dont use
- **aug_ops:** String - what augmentation procedure do you want to run
- **aug_copy:** Integer - how many augmentations per example are to be generated
- **uda_coeff:** Float - default 1 - This is the coefficient on the UDA loss. Basically you can rely more or less on the UDA loss during the supervised training. The UDA paper generally kept this at 1
- **tsa:** String - Annealing schedule to use. Options provided are "" none, linear_schedule, log_schedule, exp_schedule
- **uda_softmax_temp:** Float, default -1, A smaller temperature will accentuate differences in probabilities. Low temps were used in the UDA paper for cases with low numbers of labeled data, after masking out uncertain predictions.
- **uda_confidence_thresh:** Float, default -1, Threshold value above which the consistency loss term from the UDA is used. Basically ensures we are using loss from random guesses.

### TPU and GPU Options:
- **use_tpu:** Boolean - self-explanatory - it affects how the model is run. If we run in colab this could be important. False means use CPU or GPU. We will default to FALSE.
- **tpu_name:** String - address of the tpu
- **gcp_project:** String - project name when using TPU
- **tpu_zone:** String - can be set or detected
- **master:** Address of the TPU master, if applicable



### Defaults - Use Bert-based or XLNet-based

The defaults below should not be changed. Note that a config file will be read in after this in order to update these if desired.

In [0]:
lm = 'XLNET' # must be BERT or XLNET

if lm == 'BERT':
    defaults_file = drive_path + '/config/defaults.yml'
    data_dir = bucket_address + '/Data/proc_data/GoT'
    warmup = 300
    steps = 3000
    lrate = 3e-5
    config_file= os.path.join(bucket_address, 'bert_pretrained', 'bert_base','bert_config.json')
    init_checkpoint =  os.path.join(bucket_address, 'bert_pretrained','bert_base','bert_model.ckpt')
    spiece_model_file = None
    vocab_file = os.path.join(bucket_address, 'bert_pretrained','bert_base','vocab.txt')
elif lm == 'XLNET':
    defaults_file = os.path.join(drive_path, 'config','defaults-xlnet.yml')
    data_dir = os.path.join(bucket_address, 'Data', 'proc_data','GoT_xlnet')
    warmup = 150
    steps = 1500
    lrate = 1e-5
    config_file= os.path.join(bucket_address, 'xlnet_pretrained', 'xlnet_base','xlnet_config.json')
    init_checkpoint =  os.path.join(bucket_address, 'xlnet_pretrained','xlnet_base','xlnet_model.ckpt')
    spiece_model_file= os.path.join(bucket_address, 'xlnet_pretrained','xlnet_base', 'spiece.model')
    vocab_file = None
else:
    raise ValueError("lm must be equal to BERT or XLNET")
      
with open(defaults_file, 'r') as def_config:
    options = yaml.safe_load(def_config)
    


## Set the Case to Run
This will ensure that different configurations are being controlled and saved separately. Just load in the correct yaml file that specifies all of the parameters.

### Updated to include UDA parameters

In [0]:
train_set = 'train_200'
batch_size = 32
hdo = 0.10
uda = True
tfidf = 3
ur = 1
udac = 1
udact = -1
cda_smt = -1

case_specifics = {'do_train': True,
'do_eval' : True,
'sup_train_data_dir': os.path.join(data_dir, train_set),
'eval_data_dir':  os.path.join(data_dir, 'dev'),
'model_dir':  os.path.join(bucket_address, lm, train_set, "bs"+str(batch_size)+"_hdo{:2.0f}".format((hdo*100))),
'model_config_file': config_file,
'init_checkpoint': init_checkpoint,
'spiece_model_file': spiece_model_file,
'vocab_file': vocab_file,
'num_train_steps':  steps, 
'train_batch_size': batch_size,
'learning_rate':  lrate,
'num_warmup_steps':  warmup, 
'hidden_dropout': hdo,
'attention_dropout': 0.1,
'save_checkpoints_num' :10,
'use_tpu' :True,
'use_one_hot_embeddings': True,
'tpu': tpu_address,
'language_model': lm}

uda_options = {   'unsup_data_dir': os.path.join(data_dir, 'unsup'), 
                  'unsup_ratio': 1,
                  'tsa' : 'exp_schedule',
                  'aug_ops': 'tf_idf-0.{}'.format(tfidf),
                  'aug_copy': 1,
                  'uda_softmax_temp': -1,
                  'uda_confidence_thresh':udact,
                  'uda_coeff': udac,
                  'model_dir': os.path.join(
                      bucket_address, lm, train_set, "uda_bs{}_hdo{:2.0f}_exp_tf{}_ur{}_lr{:1.0f}".format(
                          batch_size, (hdo*100), tfidf, ur, lrate*1e5))}

In [0]:
# merge dictionaries    
options.update(case_specifics)
if uda:
  options.update(uda_options)

#
print()
print("="*50 + "\nFull Listing of Options: \n" + "="*50)
pprint.pprint(options)


Full Listing of Options: 
{'adam_epsilon': 1e-08,
 'attention_dropatt': 0.1,
 'attention_dropout': 0.1,
 'aug_copy': 1,
 'aug_ops': 'tf_idf-0.3',
 'clamp_len': -1,
 'clip_norm': 1.0,
 'cls_scope': False,
 'data_dir': './Data',
 'decay_method': 'poly',
 'do_eval': True,
 'do_predict': False,
 'do_train': True,
 'eval_all_ckpt': True,
 'eval_batch_size': 8,
 'eval_data_dir': 'gs://nlp_finalproject/Data/proc_data/GoT_xlnet/dev',
 'eval_split': 'dev',
 'gcp_project': False,
 'hidden_dropout': 0.1,
 'init': 'normal',
 'init_checkpoint': 'gs://nlp_finalproject/xlnet_pretrained/xlnet_base/xlnet_model.ckpt',
 'init_range': 0.1,
 'init_std': 0.02,
 'is_regression': False,
 'iterations_per_loop': 1000,
 'language_model': 'XLNET',
 'learning_rate': 1e-05,
 'lr_layer_decay_rate': 1.0,
 'master': False,
 'max_save': 5,
 'max_seq_length': 128,
 'min_lr_ratio': 0.0,
 'model_config_file': 'gs://nlp_finalproject/xlnet_pretrained/xlnet_base/xlnet_config.json',
 'model_dir': 'gs://nlp_finalproject/XLNET

## Setup the Job
This section of the code grabs the right data and reads in the BERT config file. We also dump our configuration options to a JSON file in the model directory.

In [0]:
# Record informational logs
logging.set_verbosity(logging.INFO)

# GoT Processor Class has methods for retrieving text/train/dev datasets
processor = raw_data_utils.GoTProcessor()

# Read in the labels
label_list = processor.get_labels()

# Create the directory for the current model
tf.io.gfile.makedirs(options['model_dir'])

if options['language_model'] == 'BERT':
    # Read the BertConfig File
    config = modeling.BertConfig.from_json_file(
          options['model_config_file'],
          options['hidden_dropout'],
          options['attention_dropout'])
    tf.io.write_file(os.path.join(options['model_dir'], "OPTIONS.json"), json.dumps(options))
elif options['language_model'] == 'XLNET':
    # construct xlnet config and save to model_dir
    config = xlnet_colab.XLNetConfig(json_path=options['model_config_file'])
    config.to_json(os.path.join(options['model_dir'], "config.json"))

## Model Specific Setup

In [0]:
logging.info("warmup steps {}/{}".format(
      options['num_warmup_steps'], options['num_train_steps']))

# Specify where the checkpoints will be saved. This is just integer division between the total number of training steps and the number of checkpoints
options['save_checkpoints_steps'] = options['num_train_steps'] // options['save_checkpoints_num']

# Log the checkpoints
logging.info("setting save checkpoints steps to {:d}".format(
      options['save_checkpoints_steps']))

# Update iterations per loop
options['iterations_per_loop'] = min(options['save_checkpoints_steps'],
                                  options['iterations_per_loop'])

INFO:absl:warmup steps 150/1500
INFO:absl:setting save checkpoints steps to 150


## Setup Run Configuration

In [0]:
# Replaced old logic with xlnet logic - should work for BERT and XLNET now
run_config = model_utils.configure_tpu(options)


INFO:tensorflow:Use TPU without distribute strategy.


INFO:tensorflow:Use TPU without distribute strategy.


## Create our model
Our model uses the bert configuration parameters as well as gets fed all of our particular running options.

In [0]:
model_fn = uda_colab.model_fn_builder(
      options,
      config=config,
      num_labels=len(label_list),
      print_feature=False,
      print_structure=False,
      freeze_layers = (False, )
  )

## Create our Estimator
Basically contains the intstructions to run the model function.

In [0]:

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=options['use_tpu'],
    model_fn=model_fn,
    config=run_config,
    params={"model_dir": options['model_dir']},
    train_batch_size=options['train_batch_size'],
    eval_batch_size=options['eval_batch_size'],
    predict_batch_size=options['eval_batch_size'])



INFO:tensorflow:Using config: {'_model_dir': 'gs://nlp_finalproject/XLNET/train_200/uda_bs32_hdo10_exp_tf3_ur1_lr1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 150, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6416dcc978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.34.225.66:8470', '_evaluation_master': 'grpc://10.34.225.66:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=150, num_shards=None, num

INFO:tensorflow:Using config: {'_model_dir': 'gs://nlp_finalproject/XLNET/train_200/uda_bs32_hdo10_exp_tf3_ur1_lr1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 150, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6416dcc978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.34.225.66:8470', '_evaluation_master': 'grpc://10.34.225.66:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=150, num_shards=None, num

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


## Ready to Train

In [0]:
# Logical check to determine if we are training (vice evaluating)
if options['do_train']:
    logging.info("Getting data for supervised training from: {}".format(options['sup_train_data_dir']))
    
    # Are we doing UDA or just simple finetuning?
    if options['unsup_ratio'] > 0:
        logging.info("Getting unsupervised data from: {}".format(
          options['unsup_data_dir']))
    
    # Pass on all of the training sup/unsup options
    train_input_fn = proc_data_utils.training_input_fn_builder(
        options['language_model'],
        options['sup_train_data_dir'],
        max_seq_len=options['max_seq_length'],
        unsup_data_base_path=options['unsup_data_dir'],
        aug_ops=options['aug_ops'],
        aug_copy=options['aug_copy'],
        unsup_ratio=options['unsup_ratio'])

INFO:absl:Getting data for supervised training from: gs://nlp_finalproject/Data/proc_data/GoT_xlnet/train_200
INFO:absl:Getting unsupervised data from: gs://nlp_finalproject/Data/proc_data/GoT_xlnet/unsup


INFO:tensorflow:loading training data from these files: gs://nlp_finalproject/Data/proc_data/GoT_xlnet/train_200/tf_examples.tfrecord.0.0


INFO:tensorflow:loading training data from these files: gs://nlp_finalproject/Data/proc_data/GoT_xlnet/train_200/tf_examples.tfrecord.0.0


INFO:tensorflow:Getting Augmented Data Files


INFO:tensorflow:Getting Augmented Data Files


...

INFO:tensorflow:exist_copy_num: 


INFO:tensorflow:Current number of files to process: 4


INFO:tensorflow:Current number of files to process: 4


INFO:tensorflow:Number of data files to process: 4


INFO:tensorflow:Number of data files to process: 4


In [0]:
# Logical check to see if we are evaluating against the development set (or test set if you change the eval_data_dir)
if options['do_eval']:
    logging.info("  >>> dev data dir : {}".format(options['eval_data_dir']))
    eval_input_fn = proc_data_utils.evaluation_input_fn_builder(
        options['language_model'],
        options['eval_data_dir'],
        "clas",
        max_seq_len=options['max_seq_length'])

    eval_size = processor.get_dev_size()
    eval_steps = int(eval_size / options['eval_batch_size'])
    logging.info("Evaluating {} examples in {} steps".format(eval_size,eval_steps))

INFO:absl:  >>> dev data dir : gs://nlp_finalproject/Data/proc_data/GoT_xlnet/dev


INFO:tensorflow:loading eval clas data from these files: gs://nlp_finalproject/Data/proc_data/GoT_xlnet/dev/tf_examples.tfrecord.0.0


INFO:tensorflow:loading eval clas data from these files: gs://nlp_finalproject/Data/proc_data/GoT_xlnet/dev/tf_examples.tfrecord.0.0
INFO:absl:Evaluating 2500 examples in 312 steps


In [0]:
# IF we are training and evaluating
eval_results = []
if options['do_train'] and options['do_eval']:
    logging.info("***** Running training & evaluation *****")
    logging.info("  Supervised batch size = {:d}".format(
        options['train_batch_size']))
    logging.info("  Unsupervised batch size = {:d}".format(
        options['train_batch_size'] * options['unsup_ratio']))
    logging.info("  Num steps = {}".format(options['num_train_steps']))
    logging.info("  Base evaluation batch size = {:d}".format(
        options['eval_batch_size']))
    logging.info("  Num steps = {:d}".format(eval_steps))
    
    # Initialize
    best_acc = 0
    
    # Looping over training steps by subset (for each checkpoint)
    for s in range(0, options['num_train_steps'], options['save_checkpoints_steps']):
        logging.info("*** Running training: Current Step: {}***".format(s))
        
        estimator.train(
              input_fn=train_input_fn,
              steps=options['save_checkpoints_steps'])
        
        logging.info("*** Running evaluation ***")
        dev_result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
        logging.info(">> Results:")
        
        # Keep track of the evaluation results
        for key in dev_result.keys():
            logging.info("  {} = {}".format(key, str(dev_result[key])))
            dev_result[key] = dev_result[key].item()
        eval_results.append(dev_result)
        # Update the best accuracy object
        best_acc = max(best_acc, dev_result["eval_classify_accuracy"])
    logging.info("***** Final evaluation result *****")
    logging.info("Best acc: {:.3f}\n\n".format(best_acc))
    
    eval_results.sort(key=lambda x: x['eval_classify_accuracy'], reverse=True)

    tf.logging.info("=" * 80)
    log_str = "Best result | "
    for key, val in sorted(eval_results[0].items(), key=lambda x: x[0]):
      log_str += "{} {} | ".format(key, val)
    tf.logging.info(log_str)
    
elif options['do_train']:
    logging.info("***** Running training *****")
    logging.info("  Supervised batch size = {}".format(options['train_batch_size']))
    logging.info("  Unsupervised batch size = {}".format(
                    options['train_batch_size'] * options['unsup_ratio']))
    logging.info("  Num steps = {}".format(options['num_train_steps']))
    estimator.train(input_fn=train_input_fn, max_steps=options['num_train_steps'])
elif options['do_eval']:
    logging.info("***** Running evaluation *****")
    logging.info("  Base evaluation batch size = {}".format(options['eval_batch_size']))
    logging.info("  Num steps = {}".format(eval_steps))
    
    # Load in the checkpoint from training to do the evaluation
    checkpoint_state = tf.train.get_checkpoint_state(options['model_dir'])

    best_acc = 0
    for ckpt_path in checkpoint_state.all_model_checkpoint_paths:
        if not tf.io.gfile.exists(ckpt_path + ".data-00000-of-00001"):
            logging.info(
                "Warning: checkpoint {:s} does not exist".format(ckpt_path))
        continue
        logging.info("Evaluating {:s}".format(ckpt_path))
        dev_result = estimator.evaluate(
          input_fn=eval_input_fn,
          steps=eval_steps,
          checkpoint_path=ckpt_path,)
        logging.info(">> Results:")
        
        # keep track of evaluation metrics
        for key in dev_result.keys():
            logging.info("  {:s} = {:s}".format(key, str(dev_result[key])))
            dev_result[key] = dev_result[key].item()
        
        # update our best accuracy variable
        best_acc = max(best_acc, dev_result["eval_classify_accuracy"])
    logging.info("***** Final evaluation result *****")
    logging.info("Best acc: {:.3f}\n\n".format(best_acc))
    
    

INFO:absl:***** Running training & evaluation *****
INFO:absl:  Supervised batch size = 32
INFO:absl:  Unsupervised batch size = 32
INFO:absl:  Num steps = 1500
INFO:absl:  Base evaluation batch size = 8
INFO:absl:  Num steps = 312
INFO:absl:*** Running training: Current Step: 0***


INFO:tensorflow:Querying Tensorflow master (grpc://10.34.225.66:8470) for TPU system metadata.


INFO:tensorflow:Querying Tensorflow master (grpc://10.34.225.66:8470) for TPU system metadata.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:evaluation_loop marked as finished
INFO:absl:>> Results:
INFO:absl:  eval_classify_accuracy = 0.33333334
INFO:absl:  eval_classify_loss = 1.4870175
INFO:absl:  eval_mpca = 0.33655646
INFO:absl:  eval_precision = 0.8924462
INFO:absl:  eval_recall = 0.8610039
INFO:absl:  loss = 1.5052801
INFO:absl:  global_step = 1500
INFO:absl:***** Final evaluation result *****
INFO:absl:Best acc: 0.345




INFO:tensorflow:================================================================================


INFO:tensorflow:================================================================================


INFO:tensorflow:Best result | eval_classify_accuracy 0.34495192766189575 | eval_classify_loss 0.7999811768531799 | eval_mpca 0.34873145818710327 | eval_precision 0.8919354677200317 | eval_recall 0.8006756901741028 | global_step 300 | loss 0.864227294921875 | 


INFO:tensorflow:Best result | eval_classify_accuracy 0.34495192766189575 | eval_classify_loss 0.7999811768531799 | eval_mpca 0.34873145818710327 | eval_precision 0.8919354677200317 | eval_recall 0.8006756901741028 | global_step 300 | loss 0.864227294921875 | 


In [0]:
eval_results

[{'eval_classify_accuracy': 0.34495192766189575,
  'eval_classify_loss': 0.7999811768531799,
  'eval_mpca': 0.34873145818710327,
  'eval_precision': 0.8919354677200317,
  'eval_recall': 0.8006756901741028,
  'global_step': 300,
  'loss': 0.864227294921875},
 {'eval_classify_accuracy': 0.3333333432674408,
  'eval_classify_loss': 1.4160550832748413,
  'eval_mpca': 0.33655646443367004,
  'eval_precision': 0.8924462199211121,
  'eval_recall': 0.8610038757324219,
  'global_step': 1050,
  'loss': 1.4210516214370728},
 {'eval_classify_accuracy': 0.3333333432674408,
  'eval_classify_loss': 1.473380208015442,
  'eval_mpca': 0.33655646443367004,
  'eval_precision': 0.8924462199211121,
  'eval_recall': 0.8610038757324219,
  'global_step': 1200,
  'loss': 1.4900565147399902},
 {'eval_classify_accuracy': 0.3333333432674408,
  'eval_classify_loss': 1.4868674278259277,
  'eval_mpca': 0.33655646443367004,
  'eval_precision': 0.8924462199211121,
  'eval_recall': 0.8610038757324219,
  'global_step': 135

In [0]:
!ls ./model/XLNET/train_5000/


ls: cannot access './model/XLNET/train_5000/': No such file or directory


In [0]:
!mv ./model/XLNET/train_5000/uda_bs12_hdo10_log_tf3_ur1/checkpoint /content/drive/My\ Drive/NLP_FinalProject/working/model/XLNET/train_5000/uda_bs12_hdo10_log_tf3_ur1_lr2_3000
!mv ./model/XLNET/train_5000/uda_bs12_hdo10_log_tf3_ur1/events* /content/drive/My\ Drive/NLP_FinalProject/working/model/XLNET/train_5000/uda_bs12_hdo10_log_tf3_ur1_lr2_3000
!mv ./model/XLNET/train_5000/uda_bs12_hdo10_log_tf3_ur1/graph* /content/drive/My\ Drive/NLP_FinalProject/working/model/XLNET/train_5000/uda_bs12_hdo10_log_tf3_ur1_lr2_3000
!mv ./model/XLNET/train_5000/uda_bs12_hdo10_log_tf3_ur1/model.ckpt-1800* /content/drive/My\ Drive/NLP_FinalProject/working/model/XLNET/train_5000/uda_bs12_hdo10_log_tf3_ur1_lr2_3000

In [0]:
!rm -rf ./model/XLNET

In [0]:
!ls /content/drive/My\ Drive/NLP_FinalProject/working/model/XLNET/train_5000/

bs32_hdo10_lr3	uda_bs12_hdo10_log_tf1_ur1_lr3_3000
bs32_hdo15	uda_bs12_hdo10_log_tf3_ur1_lr2_3000
